# 1. Load Environment & Huggingface, W&B login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_03"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Dataset Load

- 이 시스템에서는 작업을 외장하드에서 하고 있기 때문에 캐쉬폴더를 별도로 지정(In this system, tasks are being performed on an external hard drive, so a separate cache folder is specified.)

In [3]:
ko_instruction_01 = load_dataset("nlpai-lab/kullm-v2", cache_dir="/mnt/t7/.cache/huggingface/datasets", split="train")

In [4]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [5]:
ko_instruction_01

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [6]:
ko_instruction_01 = ko_instruction_01.train_test_split(test_size=0.9)

In [7]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 15263
    })
    test: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 137367
    })
})

In [8]:
# ko_instruction_01 = ko_instruction_01['train'].train_test_split(test_size=0.9)

# 3. Model Load

- QLoRA로 finetune을 하기위해 일반 7B 모델을 BitsAndBytes를 통해 양자화(4bit quantization)하여 로드, 훈련속도 향상을 위해 데이터 타입을 fp32가 아니라 fp16으로 로드(차후에는 bf16으로 로드해서 훈련예정)
- To finetune with QLoRA, a standard 7B model is quantized (4-bit quantization) through BitsAndBytes for loading, and to improve training speed, the data type is loaded as fp16 instead of fp32 (with plans to load and train with bf16 in the future).


```python

quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
```

### *Reference

- https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py
- https://colab.research.google.com/drive/19AFEOrCI6-bc7h9RTso_NndRwXJRaJ25?usp=sharing

In [9]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [10]:
checkpoint_dir = "./checkpoint/experi_03"
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}

In [11]:
# 4bit QLoRA 학습을 위한 설정
bnb_4bit_compute_dtype = "bfloat16" # 코랩 무료버전에서 실행 시 "float16"를 사용하세요
bnb_4bit_quant_type = "nf4"
use_4bit = True
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device_map, cache_dir="/mnt/t7/.cache/huggingface/models")

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

### 1) PEFT훈련을 위한 CONFIG설정, 이 부분은 Trainable Parameter를 확인하지 않아도 QLoRA를 위해서는 무조건 해야 한다.("For PEFT training configuration, this step must be done unconditionally for QLoRA, even without checking for Trainable Parameters.")

In [13]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    inference_mode=False,
    task_type="CAUSAL_LM",
)

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/mnt/t7/.cache/huggingface/models")

tokenizer.pad_token = tokenizer.eos_token

# # 학습 진행 중 loss가 치솟다가 0.0으로 떨어지는 문제 해결을 위해 사용
tokenizer.padding_side = "right"

### 4) Prompt 설정(Prompt configuration)

- Prompt는 아래와 같음(The prompt is as follows.)
> \#\#\# System: content  
> \#\#\# Human: content  
> \#\#\# Assistant: content  

- Prompt에 맞춰서 Dataset을 재가공(Reprocessing the Dataset according to the Prompt)

In [15]:
def format_instruction(sample):
    system_prompt = f"### instruction: {sample['instruction']}"
    input = f"### input: {sample['input']}" if len(sample["input"]) > 0 else None
    output = f"### output: {sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [16]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/15263 [00:00<?, ? examples/s]

In [17]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 15263
})

In [18]:
train_dataset["text"][1000]

'### instruction: 주어진 데이터 세트에 대해 적절한 데이터 정리 기술을 수행합니다.\n\n### input: "국가", "도시", "인구" 열이 있는 데이터 집합\n\n### output: \'국가\', \'도시\', \'인구\' 열이 있는 주어진 데이터 집합에 대해 적절한 데이터 정리 기법을 수행하려면 다음 단계를 수행할 수 있습니다:\n\n1. **누락된 데이터**: 세 열 모두에서 누락된 데이터가 있는지 확인합니다. 빈 셀이 있는 경우 전체 행을 삭제하거나 데이터 집합에 대한 이해를 바탕으로 누락된 데이터를 적절한 값으로 채울 수 있습니다.\n\n2. **데이터 유형**: 각 열의 데이터 유형이 적절한지 확인합니다. 예를 들어, \'국가\' 및 \'도시\' 열의 데이터 유형은 문자열 또는 텍스트여야 하고 \'인구\'는 정수 또는 실수여야 합니다.\n\n3. **중복 데이터**를 클릭합니다: 데이터 집합에 중복되는 행이 있는지 확인하고 제거합니다.\n\n4. **일관성**: 열의 데이터가 일관성이 있는지 확인합니다. 예를 들어, \'국가\' 열에 전체 국가명이 포함된 경우 모든 행에서 국가명이 동일해야 합니다. \'도시\' 열도 마찬가지입니다.\n\n5. **이상값**: 분석에 오류를 일으킬 수 있는 \'인구\' 열에 이상값이 있는지 확인합니다. 통계적 방법이나 시각화 기법을 사용하여 이상값을 식별하고 제거해야 하는지 여부를 결정할 수 있습니다.\n\n6. **표준화**: 추가 분석 또는 모델링에 데이터를 사용해야 하는 경우, 요구사항에 따라 \'모집단\' 열의 값을 표준화하거나 정규화해야 할 수 있습니다.\n\n위의 단계를 수행한 후에는 추가 분석에 사용할 수 있는 깨끗한 데이터 세트가 준비되어 있어야 합니다.</s>'

In [19]:
train_dataset["text"][3000]

'### instruction: 전문 사진 작가처럼 행동하십시오. Amazon FBA 창고의 이미지를 캡처하는 프롬프트를 만들어야 합니다. 카메라 설정을 포함하세요. 영어로 작성하세요.\n\n### output: rompt: 아마존 FBA(아마존 주문 처리) 창고의 방대한 규모와 정돈된 혼란스러움을 보여주는 인상적인 이미지를 캡처하세요. 사진에는 엄청난 수의 제품, 복잡한 물류 시스템, 운영의 효율성이 강조되어야 합니다.카메라 설정:1. 카메라: Canon EOS 5D Mark IV 또는 이와 유사한 풀프레임 DSLR2. 렌즈: 광활한 창고를 담을 수 있는 Canon EF 16-35mm f/2.8L III USM 또는 동급의 광각 줌 렌즈3. ISO: 400-800(창고 내 사용 가능한 조명에 따라 다름)4. 셔터 속도: 1/100 - 1/200초(움직이는 작업자와 장비의 모션 블러를 방지하기 위해)5. 조리개: f/5.6 - f/8, 피사계 심도를 깊게 하여 창고 대부분에 초점 맞추기6. 삼각대: 필요한 경우 장노출 촬영 시 카메라를 안정화할 수 있는 견고한 삼각대7. 원격 셔터 릴리즈: 촬영 중 카메라 흔들림을 제거하기 위해컨베이어 벨트나 보관 선반과 같은 주요 선으로 샷을 구성하여 시청자의 시선을 이미지로 유도합니다. 움직이는 작업자, 제품을 운반하는 지게차, 작업 중인 로봇 시스템을 포함하여 역동적인 환경을 포착합니다. 사용 가능한 자연광이나 오버헤드 인공 조명을 활용하여 장면을 밝히고, 조명 조건이 좋지 않은 경우 HDR 이미지를 만들기 위해 브라케팅 노출을 사용하는 것도 고려하세요.최종 사진은 아마존 FBA 창고의 인상적인 규모와 복잡성을 전달하는 동시에 효율적이고 능률적인 운영 특성을 보여줄 수 있어야 합니다.</s>'

In [20]:
train_dataset["text"][3230]

'### instruction: 예, 계속\n\n### output: 각 개념에 대한 몇 가지 추가 연습 문제가 있습니다:수식 구문 이해를 위한 연습:* 다음 수식에서 구문 요소를 식별하세요:text(month(today()) & "/" & text(day(today()) & "/" & text(year(today()))간단한 수식 만들기 연습:* 연락처 개체에서 생년월일을 기준으로 연락처의 나이를 계산하는 수식 필드를 만듭니다.복잡한 수식 작성 연습:* 기회 단계가 "마감 원"으로 설정되어 있지 않은 경우 마감 날짜가 기회 생성 날짜로부터 최소 7일이 지나야 한다는 유효성 검사 규칙을 기회 개체에 만듭니다.공식 디버깅 연습:* 다음 수식에서 오류를 식별하고 수정된 버전을 제공하세요:IF(스테이지 = "유망" AND 금액 > 10000, "고가", "저가")수식 필드 사용 연습:* 리드 개체에서 여러 사용자 지정 필드의 값을 기반으로 리드 점수를 계산하는 수식 필드를 만듭니다.유효성 검사 규칙 사용 연습:* 연락처 개체에서 이메일 또는 전화 필드를 입력해야 하는 유효성 검사 규칙을 만듭니다.보고서 수식 사용 연습:* 계정 보고서에서 각 계정과 연결된 총 연락처 수를 계산하는 보고서 수식을 만듭니다.고급 수식 기법을 위한 연습:* 케이스 객체에서 케이스 생성 날짜와 가장 최근의 케이스 댓글 생성 날짜 사이의 일 수를 표시하는 수식 필드를 만듭니다.수식 작성 모범 사례 연습:* 기존 수식 필드 또는 유효성 검사 규칙을 검토하고 수식을 단순화하거나 최적화할 수 있는 기회를 파악합니다.</s>'

In [21]:
train_dataset["text"][12312]

'### instruction: 초\\_total` 및 `밀리초`를 계산할 때 `uint32\\_t`로 명시적으로 캐스팅해야 하나요? 아니면 그냥 좋은 스타일인가요?\n\n### output: 이 경우 `uint32_t`로 명시적 형변환이 반드시 필요한 것은 아닙니다. uint64_t`인 `timestamp` 변수에 대한 나누기 및 모듈로 연산은 할당되는 32비트 변수에 맞게 자동으로 잘립니다.그러나 명시적 형변환을 사용하면 프로그래머의 의도를 명확히 하고 값이 의도적으로 32비트 값으로 잘린다는 것을 명확히 할 수 있으므로 좋은 스타일로 간주할 수 있습니다. 오버플로 또는 예기치 않은 동작이 발생할 가능성이 있는 다른 경우에는 잠재적인 문제를 피하기 위해 명시적 형변환이 중요할 수 있습니다.요약하면, 명시적 형변환이 반드시 필요한 것은 아니지만 코드를 더 읽기 쉽게 만들고 프로그래머의 의도를 전달하는 데 도움이 될 수 있습니다.</s>'

In [22]:
# test_dataset = ko_instruction_01['test'].map(template_dataset, remove_columns=list(ko_instruction_01['test'].features), num_proc=10)

# 4. Configure Train arguments & Define Trainer

In [23]:
training_arguments = TrainingArguments(
    output_dir=checkpoint_dir,
    save_steps=200,
    save_total_limit=3, # 가장 최근 체크포인트 3개만 저장합니다.
    logging_steps=30,
    report_to="wandb",
    learning_rate=3e-4,
    max_grad_norm=0.3,
    num_train_epochs=2, # epochs 대신 max_steps을 기준으로 할 수 있습니다.
    warmup_ratio=0.02,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit", # paged_adamw_8bit 사용시 메모리를 더 절약할 수 있지만 loss가 0으로 떨어지는 문제가 있습니다.
    group_by_length=True,
    fp16 = False, # 코랩 무료버전에서 실행 시 "True"를 사용하세요
    bf16 = True, # 코랩 무료버전에서 실행 시 "False"를 사용하세요
    lr_scheduler_type="constant",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/15263 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=1024,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=False,
# )

In [25]:
trainer.train()


  0%|          | 0/30526 [00:00<?, ?it/s]

{'loss': 2.2808, 'grad_norm': 0.400390625, 'learning_rate': 0.0003, 'epoch': 0.0}
{'loss': 1.8199, 'grad_norm': 0.80078125, 'learning_rate': 0.0003, 'epoch': 0.0}
{'loss': 1.9828, 'grad_norm': 0.54296875, 'learning_rate': 0.0003, 'epoch': 0.01}
{'loss': 1.8847, 'grad_norm': 0.29296875, 'learning_rate': 0.0003, 'epoch': 0.01}
{'loss': 2.031, 'grad_norm': 1.25, 'learning_rate': 0.0003, 'epoch': 0.01}
{'loss': 1.9026, 'grad_norm': 0.392578125, 'learning_rate': 0.0003, 'epoch': 0.01}
{'loss': 2.0661, 'grad_norm': 0.26171875, 'learning_rate': 0.0003, 'epoch': 0.01}
{'loss': 1.8414, 'grad_norm': 0.7109375, 'learning_rate': 0.0003, 'epoch': 0.02}
{'loss': 1.8244, 'grad_norm': 0.41796875, 'learning_rate': 0.0003, 'epoch': 0.02}
{'loss': 1.8273, 'grad_norm': 2.15625, 'learning_rate': 0.0003, 'epoch': 0.02}
{'loss': 1.9557, 'grad_norm': 0.365234375, 'learning_rate': 0.0003, 'epoch': 0.02}
{'loss': 1.8285, 'grad_norm': 0.22265625, 'learning_rate': 0.0003, 'epoch': 0.02}
{'loss': 1.7757, 'grad_nor

TrainOutput(global_step=30526, training_loss=1.6338039096526609, metrics={'train_runtime': 15595.4896, 'train_samples_per_second': 1.957, 'train_steps_per_second': 1.957, 'train_loss': 1.6338039096526609, 'epoch': 2.0})

### 허깅페이스에서 훈련 도중 러닝레이트가 수정 안되는 경우가 많다 한다. 
- 스케쥴러 변경을 시도해 보기로 한다. 

In [28]:
output_dir = "./results/experi_03"

In [29]:
trainer.model.save_pretrained(output_dir)